In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from gl_functions import *
from spot_functions import *
from constants import timeout_sec
from scipy.spatial.transform import Rotation as sciRot

In [3]:
#### CONSTANTS ####
look_at_locs = np.array([[1.1, 0, -0.5],[0, 1.1, -0.5]])

obj_pos_body = geometry_pb2.Vec3(x = 0.8, y = 0, z = -0.55)

arm_pos_bounds  = {'x': {'lower':  0.6, 'upper': 0.9},
                   'y': {'lower': -0.4, 'upper': 0.4},
                   'z': {'lower': -0.1, 'upper': 0.8}}

drop_rots = np.array([
    [1,0,0],
    [-1,0,0],
    [1,0,1],
    [-1,0,1],
    [1,0,-1],
    [-1,0,-1],
    [0,1,0],
    [0,1,1],
    [0,1,2],
    [0,1,3]
])

upload_filepath = "/home/gluser/groundlight/bdspot/spot/loopGraph"

clear_waypoints = np.array([9, 10, 12, 25])

In [4]:
drop_rots.shape[0]

10

In [6]:
sdk, robot, id_client, \
robot_state_client, command_client, \
image_client, graph_nav_client, \
world_object_client, manipulation_api_client = init_robot('192.168.50.3')

In [7]:
lease_client, lease, lease_keep_alive = get_lease(robot)

In [8]:
# Initialize Groundlight API

GROUNDLIGHT_API_TOKEN = 'api_2Au8NPiTMbSLQ73Vjr2eMhX2qUh_pt4EEKhtDNgnjN3dUAr89YANDcYsmBkJZV'
gl = Groundlight(api_token=GROUNDLIGHT_API_TOKEN)
detectors = gl.list_detectors()
eraser_det = detectors.results[0]

In [9]:
rubiks_det = detectors.results[4]

In [10]:
get_batt_info(robot_state_client)

Battery at: 81.0 %. Robot has 4567 seconds left


(81.0, 4567)

In [32]:
success = power_on(robot)

In [33]:
bdcrc.blocking_selfright(command_client, timeout_sec = 20)

In [14]:
bdcrc.blocking_stand(command_client, timeout_sec = timeout_sec)

In [23]:
move_robot_relative(command_client, robot, 0, -1, 0)

In [15]:
def gripper_slow_open(command_client):
    
    gripper_angles = np.linspace(0, 1, 1000)
    
    for gripper_angle in gripper_angles:
        cmd = RobotCommandBuilder.claw_gripper_open_fraction_command(gripper_angle)
        cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+timeout_sec)
        time.sleep(1/gripper_angles.shape[0])

In [16]:
def random_rot_drop(command_client, manipulation_api_client):
    drop_rot_ind = np.random.randint(drop_rots.shape[0])
    drop_rot = drop_rots[drop_rot_ind, :]
    
    rotObj = sciRot.from_euler('xyz',np.pi/2*drop_rot)
    rot_mtx = rotObj.as_matrix()
    rot_quat = bosdyn.client.math_helpers.Quat.from_matrix(rot_mtx)
    drop_pose_body_frame_proto = geometry_pb2.SE3Pose(
        position = geometry_pb2.Vec3(x = 0.8, y = 0, z = -0.3),  
        rotation = rot_quat.to_proto())
    drop_pose_body_frame_se3 = bosdyn.client.math_helpers.SE3Pose.from_proto(drop_pose_body_frame_proto)
    arm_cmd = RobotCommandBuilder.arm_pose_command(x = drop_pose_body_frame_se3.x,
                                               y = drop_pose_body_frame_se3.y,
                                               z = drop_pose_body_frame_se3.z,
                                               qw = drop_pose_body_frame_se3.rotation.w,
                                               qx = drop_pose_body_frame_se3.rotation.x,
                                               qy = drop_pose_body_frame_se3.rotation.y,
                                               qz = drop_pose_body_frame_se3.rotation.z,
                                               frame_name = 'body')

    body_cmd = RobotCommandBuilder.synchro_stand_command(body_height = -0.3)
    cmd = RobotCommandBuilder.build_synchro_command(arm_cmd, body_cmd)
    cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+timeout_sec)
    bdcrc.block_until_arm_arrives(command_client, cmd_id, timeout_sec = time.time() + 3.0)
    time.sleep(0.5)
    
    gripper_slow_open(command_client)
    time.sleep(0.5)

    body_cmd = RobotCommandBuilder.synchro_stand_command(body_height = 0)
    cmd_id = command_client.robot_command(body_cmd, end_time_secs = time.time()+timeout_sec)
    time.sleep(1)

    success = blocking_stow(command_client, timeout_sec = 3)
    override_response = stowable_override(manipulation_api_client)
    time.sleep(2)

In [17]:
def rand_imgs_thru_det(command_client, image_client, gl, det):
    for j in range(len(look_at_locs)):
        look_at = look_at_locs[j]
        for i in range(3):
            #Move Arm to random position
            x_pos_rand = np.random.uniform(arm_pos_bounds['x']['lower'],arm_pos_bounds['x']['upper'])
            y_pos_rand = np.random.uniform(arm_pos_bounds['y']['lower'],arm_pos_bounds['y']['upper'])
            z_pos_rand = np.random.uniform(arm_pos_bounds['z']['lower'],arm_pos_bounds['z']['upper'])

            move_to = np.array([x_pos_rand, y_pos_rand, z_pos_rand])
            move_to_and_look_at(command_client, move_to, look_at, blocking = True)

            time.sleep(0.4)

            hand_color_img, img_response = cap_hand_image(image_client, source = 'hand_color_image')
            image_query = mat_thru_det(gl, det, hand_color_img)

In [18]:
def multiAttemptGrasp(*, manipulation_api_client, command_client, graph_nav_client, obj_pos_vision):
    for i in range(5):
        print(i)
        
        grasp_params = manipulation_api_pb2.GraspParams(grasp_palm_to_fingertip = 0)
        grasp = manipulation_api_pb2.PickObject(frame_name = 'vision',
                                                object_rt_frame = obj_pos_vision,
                                                grasp_params = grasp_params)
        
        grasp_request = manipulation_api_pb2.ManipulationApiRequest(pick_object=grasp)
        response, success = make_grasp(grasp_request, manipulation_api_client, 
                                       command_client, graph_nav_client, verbose = True)

        curState = robot_state_client.get_robot_state()
        wrist_pos_rads = np.abs(curState.kinematic_state.joint_states[-1].position.value)

        if wrist_pos_rads < np.pi/36 or not success:
#            print(response, success)
            blocking_stow(command_client, timeout_sec = 15)
            time.sleep(3)
            continue
        else:
            return True

            
            print('GRIPPER IS CLOSED. FAILED TO GRAB CUBE')
        
    logging.warning('FAILED TO GRAB CUBE')
    return False

In [20]:
def drop_cap_grasp(*, manipulation_api_client, command_client, image_client, robot_state_client, gl, det):
    random_rot_drop(command_client, manipulation_api_client)

    #get_a_tform_b returns transformation such that a_coords = tform * b_coords
    frame_tree_snapshot = robot.get_frame_tree_snapshot()
    body_to_vision = bosdyn.client.frame_helpers.get_a_tform_b(frame_tree_snapshot, frame_a = 'vision', frame_b ='body')
    obj_pos_body_se3_proto = geometry_pb2.SE3Pose(
        position = obj_pos_body,
        rotation = bosdyn.client.math_helpers.Quat(w = 0, 
                                               x = 0, 
                                               y = -(1/2)**0.5, 
                                               z = (1/2)**0.5).to_proto())
    obj_pos_body_se3 = bosdyn.client.math_helpers.SE3Pose.from_proto(obj_pos_body_se3_proto)
    obj_pos_vision_se3 = body_to_vision * obj_pos_body_se3

    move_robot_relative(command_client, robot, -0.3, 0, 0)

    rand_imgs_thru_det(command_client, image_client, gl, rubiks_det)

    blocking_stow(command_client)

    multiAttemptGrasp(manipulation_api_client = manipulation_api_client,
                      command_client = command_client,
                      graph_nav_client = graph_nav_client,
                      obj_pos_vision = obj_pos_vision_se3.position)

    cmd = RobotCommandBuilder.arm_stow_command()
    cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+timeout_sec)

In [26]:
gripper_open(command_client)

In [137]:
gripper_close(command_client)

In [34]:
    upload_graph_and_snapshots(graph_nav_client, upload_filepath)
    success = graph_localize_fiducial(command_client, graph_nav_client, robot_state_client, robot)
    waypoint_list = create_waypoint_list(graph_nav_client)

In [27]:
for i in range(4):
    drop_cap_grasp(manipulation_api_client = manipulation_api_client,
               command_client = command_client,
               image_client = image_client,
               robot_state_client = robot_state_client,
               gl = gl,
               det = rubiks_det)

    upload_graph_and_snapshots(graph_nav_client, upload_filepath)
    success = graph_localize_fiducial(command_client, graph_nav_client, robot_state_client, robot)
    waypoint_list = create_waypoint_list(graph_nav_client)
    
    wp_ind = np.random.choice(clear_waypoints)
    print(wp_ind)
    nav_to_cmd_id = graph_nav_client.navigate_to(waypoint_list[wp_ind], cmd_duration = 15)
    wait_until_finished_nav(nav_to_cmd_id)
    
    wp_loc_proto = graph_del_get_graph_loc(graph_nav_client)

0
1


KeyboardInterrupt: 

In [36]:
    nav_to_cmd_id = graph_nav_client.navigate_to(waypoint_list[25], cmd_duration = 15)
    wait_until_finished_nav(graph_nav_client, nav_to_cmd_id)

0


9


In [114]:
success = multiAttemptGrasp(manipulation_api_client = manipulation_api_client, 
                            command_client = command_client, 
                            graph_nav_client = graph_nav_client, 
                            obj_pos_vision = obj_pos_vision_se3.position)

NameError: name 'obj_pos_vision_se3' is not defined

In [81]:
multiAttemptGrasp(manipulation_api_client, command_client, graph_nav_client)

0


TypeError: Parameter to MergeFrom() must be instance of same class: expected bosdyn.api.Vec3 got str.

In [170]:
cmd = RobotCommandBuilder.arm_stow_command()
cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+timeout_sec)

In [ ]:
move()

In [86]:
logging.getLogger().setLevel(logging.WARNING)

In [46]:
nav_to_cmd_id = graph_nav_client.navigate_to(waypoint_list[8], cmd_duration = 15)

In [26]:
move_to = np.array([0.8, 0,-0.4])
look_at = np.array([0.8, 0, -3])

In [27]:
move_to_and_look_at(command_client, move_to, look_at)

In [90]:
def get_walking_params(max_linear_vel, max_rotation_vel):
    max_vel_linear = geometry_pb2.Vec2(x=max_linear_vel, y=max_linear_vel)
    max_vel_se2 = geometry_pb2.SE2Velocity(linear=max_vel_linear,
                                           angular=max_rotation_vel)
    vel_limit = geometry_pb2.SE2VelocityLimit(max_vel=max_vel_se2)
    params = RobotCommandBuilder.mobility_params()
    params.vel_limit.CopyFrom(vel_limit)
    return params

In [130]:
params = get_walking_params(2,1.55)

In [183]:
cmd = RobotCommandBuilder.battery_change_pose_command()
cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+timeout_sec)

In [19]:
move_robot_relative(command_client, robot, 1, 0, 0)

In [17]:
move_robot_relative

<function spot_functions.move_robot_relative(command_client, robot, x_meters, y_meters, theta_rad, timeout_sec=10, blocking=True)>